<img src="./images/DLI_Header.png" style="width: 400px;">

# Assessment

Congratulations on going through today's course! Hopefully you've learned some valuable skills along the way. Now it's time to put those skills to the test. In this assessment you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of 92% in order to pass the assessment, though we challenge you to do even better if you can. You will have the use the skills that you learned in the previous exercises. Specifically we suggest you use some combination of transfer learning, data augmentation, and fine tuning. Once you have trained the model to be at least 92% accurate on the test dataset, you will save your model, and then assess its accuracy. Let's get started! 

## The Dataset

In this exercise, you will train a model to recognize fresh and rotten fruits. The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification), a great place to go if you're interested in starting a project after this class. If you'd like, you can look at the dataset structure in the `fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

<img src="./images/fruits.png" style="width: 600px;">

## Load ImageNet Base Model

We encourage you to start with a model pretrained on ImageNet. You'll need to load the model with the correct weights, set an input shape, and choose to remove the last layers of the model. Remember that images have three dimensions: a height, and width, and a number of channels. Because these pictures are in color, there will be three channels for red, green, and blue. We've filled in the input shape for you. This cannot be changed or the assessment will fail. If you need a reference for setting up the pretrained model, please take a look at notebook 05b where you first implemented transfer learning.

In [1]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

## Freeze Base Model

Next, we suggest you freeze the base model, as you did in notebook 05b. This is done so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [2]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

Now it's time to add layers to your pretrained model. You can again use notebook 05b as a guide. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.

In [3]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs,outputs)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [5]:
model.compile(loss = 'categorical_crossentropy' , metrics = ['accuracy'])

## Augment the Data

If you'd like, try to augment the data to improve the dataset. Feel free to look at notebook 04a and notebook 05b for augmentation examples. You can also look at the documentation for the [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class). This step is optional, but you may find it helpful to get to 92% accuracy when you train.

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically

## Load Dataset

Now it's time to load the train and test datasets. You'll have to pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model you've created). If you'd like a reference, you can check out notebook 05b.

In [7]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('fruits/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate test dataset
test_it = datagen.flow_from_directory('fruits/test/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `test` iterators into the `fit` function, as well as setting your desired number of epochs.

In [ ]:
model.fit(train_it,
          validation_data=test_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=test_it.samples/test_it.batch_size,
          epochs=20)

Epoch 1/20
37/36 [==============================] - 24s 641ms/step - loss: 2.4895 - accuracy: 0.4890 - val_loss: 1.1940 - val_accuracy: 0.6535
Epoch 2/20
37/36 [==============================] - 22s 602ms/step - loss: 0.7272 - accuracy: 0.7606 - val_loss: 0.6328 - val_accuracy: 0.8116
Epoch 3/20
37/36 [==============================] - 22s 602ms/step - loss: 0.3701 - accuracy: 0.8748 - val_loss: 0.4392 - val_accuracy: 0.8754
Epoch 4/20
37/36 [==============================] - 22s 601ms/step - loss: 0.2432 - accuracy: 0.9171 - val_loss: 0.2803 - val_accuracy: 0.9027
Epoch 5/20
37/36 [==============================] - 22s 603ms/step - loss: 0.1759 - accuracy: 0.9340 - val_loss: 0.2604 - val_accuracy: 0.9210
Epoch 6/20
37/36 [==============================] - 22s 601ms/step - loss: 0.1273 - accuracy: 0.9577 - val_loss: 0.2291 - val_accuracy: 0.9301
Epoch 7/20
37/36 [==============================] - 22s 601ms/step - loss: 0.1108 - accuracy: 0.9577 - val_loss: 0.1386 - val_accuracy: 0.9453

## Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest you try fine tuning the model with a very low learning rate. You may again use notebook 05b as a reference.

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model.fit(train_it,
          validation_data=test_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=test_it.samples/test_it.batch_size,
          epochs=20)

## Evaluate the Model

Hopefully you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation. 

Once you are satisfied with the validation accuracy, you can evaluate the model by executing the following cell. The evaluate function will return a tuple, where the first value is your loss, and the second value is your accuracy. You'll want to have an accuracy value of 92% or higher. 

In [ ]:
model.evaluate(test_it, steps=test_it.samples/test_it.batch_size)

## Run the Assessment

To assess your model run the following two cells.

**NOTE:** `run_assessment` assumes your model is named `model` and your test data iterator is called `test_it`. If for any reason you have modified these variable names, please update the names of the arguments passed to `run_assessment`.

In [ ]:
from run_assessment import run_assessment

In [ ]:
run_assessment(model, test_it)

## Generate a Certificate

If you passed the assessment, please return to the course page (shown below) and click the "ASSESS TASK" button, which will generate your certificate for the course.

<img src="./images/assess_task.png" style="width: 800px;">